In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [ ]:
variable_names = ['10yr', '30yr', '60yr']

In [ ]:
# out put the forced trend and pvalue
dir_in ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/CanESM5/trend/'

trend_annual_da = {}
p_val_annual_da = {}

for var in variable_names:
    trend_annual_da[var] = xr.open_mfdataset(dir_in + 'CanESM5_forced_' + var + '_trend.nc', chunks={'run': 1})
    p_val_annual_da[var] = xr.open_mfdataset(dir_in + 'CanESM5_forced_' + var + '_pvalue.nc', chunks={'run': 1})


In [ ]:
# input the unforced trend and pvalue
unforced_trend_da = {}
unforced_p_val_da = {}

for var in variable_names:
    unforced_trend_da[var] = xr.open_mfdataset(dir_in + 'CanESM5_unforced_' + var + '_trend.nc', chunks={'run': 1})
    unforced_p_val_da[var] = xr.open_mfdataset(dir_in + 'CanESM5_unforced_' + var + '_pvalue.nc', chunks={'run': 1})

In [ ]:
unforced_trend_da

### calculate the pattern correlation

In [ ]:
# Input the 10-year, 30-year and 60-year ensemble mean SAT trend data
dir_input = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1_Forced/data/Smiles_ensemble/'

CanESM5_10yr_trend   = xr.open_dataset(dir_input + 'CanESM5_annual_forced_10yr_trend.nc')
CanESM5_30yr_trend   = xr.open_dataset(dir_input + 'CanESM5_annual_forced_30yr_trend.nc')
CanESM5_60yr_trend   = xr.open_dataset(dir_input + 'CanESM5_annual_forced_60yr_trend.nc')

In [ ]:
CanESM5_10yr_trend = CanESM5_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
CanESM5_30yr_trend = CanESM5_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
CanESM5_60yr_trend = CanESM5_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

In [ ]:
trend_annual_da['10yr'].trend.isel(run=1)

In [ ]:
# pattern correlation betwenn observed forced pattern vs. Model simulated forced pattern
import scipy.stats as stats
# 10yr forced trend in each realization .vs. CanESM5 ensemble trend CanESM5_10yr_trend
CanESM5_forced_10yr_trend = trend_annual_da['10yr'].trend
CanESM5_forced_30yr_trend = trend_annual_da['30yr'].trend
CanESM5_forced_60yr_trend = trend_annual_da['60yr'].trend

CanESM5_unforced_10yr_trend = unforced_trend_da['10yr'].trend
CanESM5_unforced_30yr_trend = unforced_trend_da['30yr'].trend
CanESM5_unforced_60yr_trend = unforced_trend_da['60yr'].trend

# Pattern correlations calculation
trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    trend_pattern_correlation_10yr.append(stats.pearsonr(CanESM5_forced_10yr_trend.isel(run=i).values.flatten(), CanESM5_10yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    trend_pattern_correlation_30yr.append(stats.pearsonr(CanESM5_forced_30yr_trend.isel(run=i).values.flatten(), CanESM5_30yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    trend_pattern_correlation_60yr.append(stats.pearsonr(CanESM5_forced_60yr_trend.isel(run=i).values.flatten(), CanESM5_60yr_trend['tas'].values.flatten())[0])
    

In [ ]:
trend_pattern_correlation_60yr

In [ ]:
# save the pattern correlation into text file
# Save to a text file
with open('pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# Input the observational forced and unforced trend wrt. CanESM5-CM6A-LR
dir_obs = '/work/mh0033/m301036/Land_surf_temp/analyses_1850_2100/Manuscript_visual_schematic/Disentangling_trend_analysis/Figure2/data/'

Obs_forced_10yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_10yr_trend.nc')
Obs_forced_30yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_30yr_trend.nc')
Obs_forced_60yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_60yr_trend.nc')

Obs_unforced_10yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_10yr_trend.nc')
Obs_unforced_30yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_30yr_trend.nc')
Obs_unforced_60yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_60yr_trend.nc')

In [ ]:
Obs_unforced_10yr_trend

In [ ]:
Obs_forced_10yr_trend = Obs_forced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_forced_30yr_trend = Obs_forced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_forced_60yr_trend = Obs_forced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

Obs_unforced_10yr_trend = Obs_unforced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_unforced_30yr_trend = Obs_unforced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_unforced_60yr_trend = Obs_unforced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})


In [ ]:
# calculate the pattern correlation between observed and model's each realization simulated forced and unforced trend
OBS_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    OBS_trend_pattern_correlation_10yr.append(stats.pearsonr(CanESM5_forced_10yr_trend.isel(run=i).values.flatten(), Obs_forced_10yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    OBS_trend_pattern_correlation_30yr.append(stats.pearsonr(CanESM5_forced_30yr_trend.isel(run=i).values.flatten(), Obs_forced_30yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    OBS_trend_pattern_correlation_60yr.append(stats.pearsonr(CanESM5_forced_60yr_trend.isel(run=i).values.flatten(), Obs_forced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
OBS_trend_pattern_correlation_10yr

In [ ]:
# Save to a text file
with open('obs_MMEM_forced_vs_CanESM5_forced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
OBS_unforced_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    OBS_unforced_trend_pattern_correlation_10yr.append(stats.pearsonr(CanESM5_unforced_10yr_trend.isel(run=i).values.flatten(), Obs_unforced_10yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    OBS_unforced_trend_pattern_correlation_30yr.append(stats.pearsonr(CanESM5_unforced_30yr_trend.isel(run=i).values.flatten(), Obs_unforced_30yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    OBS_unforced_trend_pattern_correlation_60yr.append(stats.pearsonr(CanESM5_unforced_60yr_trend.isel(run=i).values.flatten(), Obs_unforced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
OBS_unforced_trend_pattern_correlation_30yr

In [ ]:
# Save to a text file
with open('obs_MMEM_unforced_vs_CanESM5_unforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")